In [1]:
import torch
from gemmi import cif
import numpy as np 
import pandas as pd
import sympy
from sympy import solve
from sympy import Symbol
import torchquad
from matplotlib import pyplot as plt
# Enable GPU support if available
torchquad.enable_cuda() 

18:52:40|TQ-INFO| Initializing torchquad.
18:52:40|TQ-INFO| __pyTorch VERSION:<module 'torch.version' from 'C:\\Users\\chemy\\Anaconda3\\lib\\site-packages\\torch\\version.py'>
18:52:40|TQ-INFO| __CUDNN VERSION:7605
18:52:40|TQ-INFO| __Number of CUDA Devices:1
18:52:40|TQ-INFO| Active CUDA Device: GPU0
18:52:40|TQ-INFO| Setting default tensor type to cuda.Float32 (CUDA is initialized).


In [2]:
def read_cif_file(file_path):
    """
    To do: given the file path, read in the cif file. 
    Input: 
        file_path: the path to the cif file
    Return: 
        cell_prams: list object contaning a,b,c,alpha, beta and gama
        atom_type: list obejct containing the types of atoms
        coor: the coordinate of the atoms
    """
    doc = cif.read_file(file_path)
    crystal = doc[0]
    cell_params = [float(crystal.find_value('_cell_length_a')),
                   float(crystal.find_value('_cell_length_b')),
                   float(crystal.find_value('_cell_length_c')),
                   float(crystal.find_value('_cell_angle_alpha'))/180*np.pi,
                   float(crystal.find_value('_cell_angle_beta'))/180*np.pi,
                   float(crystal.find_value('_cell_angle_gamma'))/180*np.pi]
    atom_type = list(crystal.find_loop('_atom_site_type_symbol'))

    coor = np.array([[float(i) for i in crystal.find_loop('_atom_site_fract_x')],
                    [float(i) for i in crystal.find_loop('_atom_site_fract_y')],
                    [float(i) for i in crystal.find_loop('_atom_site_fract_z')]]).T
    return cell_params,atom_type,coor
def read_atomic_scattering_factor(name_path,file_path):
    """
    To do: given the file path, read in the atomic scattering factors. 
           Check http://lampx.tugraz.at/~hadley/ss1/crystaldiffraction/atomicformfactors/formfactors.php for more details. 
    Input: 
        name_path: the path recording the names of the atom types
        file_path: the path recording the corresponding coefficients to calculate the scattering factor. The columns are arranged
        in the order of a1,b1,a2,b2,a3,b3,c. ()
    Return:
        sf_names: the atom names for the calculation of scattering factors
        sf_factor: the coefficeint to calculate scattering factor
    
    """
    sf_names = [i.split() for i in open(name_path).readlines()]
    sf_names = np.array(sf_names).flatten().tolist()
    
    data = pd.read_excel(file_path,header=None)
    sf_factor = np.array(data)
    
    return sf_names, sf_factor 

def calculate_fj(k,atom_type,sf_names,sf_factor):
    """
    To do: given the scattering vectors, names and coefficients, calculate the fj constant for PXRD simulation
    Input:
        k: the scattering vector defined as 4*pi/lambda*sin(theta)
        atom_type: the atom types
        sf_names: the atoms of scattering 
        sf_factor: the coefficeints for scattering calculation
    Return:
        fj: the calculated scattering factor
    """
    fj = []
    index_temp = [sf_names.index(i) for i in atom_type]
    for k_temp in k:
        for i in index_temp:
            coeff = sf_factor[i]
            part1 = coeff[0]*np.exp(-coeff[1]*((k_temp/np.pi)**2))
            part2 = coeff[2]*np.exp(-coeff[3]*((k_temp/np.pi)**2))
            part3 = coeff[4]*np.exp(-coeff[5]*((k_temp/np.pi)**2)) + coeff[5]
            fj_temp = part1 + part2+ part3
            fj.append(fj_temp)
    fj = np.array(fj).reshape((len(k),len(atom_type)))
    return fj
def solve_lattice_c(lattice_a,lattice_b):
    """
    To do: given the lattice constant, create the basic vector representing lattice c 
    Input:
        lattice_a: the lattice vector for a
        lattice_b: the lattice vector for b
    Return:
        A unit vector for lattice vector c
    """
    lattice_cx = Symbol('lattice_cx')
    lattice_cy = Symbol('lattice_cy')
    lattice_cz = Symbol('lattice_cz')
    eq1 = sympy.Eq(lattice_cx**2+lattice_cy**2+lattice_cz**2,1)
    eq2 = sympy.Eq((lattice_cx*lattice_a[0] + lattice_cy*lattice_a[1]+lattice_cy*lattice_a[2])/cell_params[0],np.cos(cell_params[4]))
    eq3 = sympy.Eq((lattice_cx*lattice_b[0] + lattice_cy*lattice_b[1]+lattice_cy*lattice_b[2])/cell_params[0],np.cos(cell_params[5]))
    result = sympy.solve([eq1,eq2,eq3],(lattice_cx,lattice_cy,lattice_cz))
    result = np.array(result).astype(float)
    
    # Only return the lattice that satisfies the right-hand relation
    for i in result:
        if i[-1]>0:
            return i
def calculate_rotation_matrix(theta):
    """
    To do: the rotation matrix for the given theta_x, theta_y, and theta_z
    Input:
    theta = [theta_x,theta_y,theta_z]
        theta_x: Rotation operation around X
        theta_y: Rotation operation around Y
        theta_z: Rotation operation around Z
    Returns:
        rot_matrix: 3x3 rotation matrix. The vector that will be rotated should be a column
    """  

    rot_x = torch.tensor([[1.0, 0.0, 0.0],
                          [0.0, torch.cos(theta[:,0]), -torch.sin(theta[:,0])],
                          [0.0, torch.sin(theta[:,0]),  torch.cos(theta[:,0])]],dtype=torch.float64)

    rot_y = torch.tensor([[ torch.cos(theta[:,1]), 0.0, torch.sin(theta[:,1])],
                          [ 0.0, 1.0, 0.0],
                          [-torch.sin(theta[:,1]), 0.0, torch.cos(theta[:,1])]],dtype=torch.float64)

#     rot_z = torch.tensor([[torch.cos(theta[:,2]), -torch.sin(theta[:,2]), 0.0],
#                           [torch.sin(theta[:,2]),  torch.cos(theta[:,2]), 0.0],
#                           [0.0, 0.0, 1.0]],dtype=torch.float64)

#     rot_matrix = torch.matmul(rot_z, torch.matmul(rot_y, rot_x))
    rot_matrix = torch.matmul(rot_y, rot_x)
    return rot_matrix

In [3]:
def calculate_intensity(theta_total):
    """
    To do: for the given rotation angle theta and the specific theta angle, calculate the diffraction intensity
    Input:
        theta: a list containing the rotation angles
        k_temp: the scattering vector
        fj_temp: the scattering factors corresponding to the scattering vector
    Output:
        I: the intensity for this specific orientation
    """
    theta_total = theta_total.reshape(-1,2)
    I = []
    for theta in theta_total:
        theta = theta.reshape(1,2)
        rot_matrix = calculate_rotation_matrix(theta)
#         rot_matrix = torch.tensor([[torch.sin(theta[:,1])*torch.cos(theta[:,0])],
#                                    [torch.sin(theta[:,0])],
#                                    [torch.cos(theta[:,1])*torch.cos(theta[:,0])]], dtype = torch.float64).flatten()
        aZ = torch.matmul(rot_matrix,lattice_a.reshape(-1,1))[-1]
        bZ = torch.matmul(rot_matrix,lattice_b.reshape(-1,1))[-1]
        cZ = torch.matmul(rot_matrix,lattice_c.reshape(-1,1))[-1]
        
        F = abs((torch.exp((coor*torch.tensor([aZ,bZ,cZ])).sum(axis=1)*k_temp*1j)*fj_temp).sum())**2
        if torch.sin(k_temp/2*aZ) == 0:
            Gx = nx**2
        else:
            Gx = (torch.sin(k_temp/2*aZ*nx)/torch.sin(k_temp/2*aZ))**2

        if torch.sin(k_temp/2*bZ) == 0:
            Gy = ny**2
        else:
            Gy = (torch.sin(k_temp/2*bZ*ny)/torch.sin(k_temp/2*bZ))**2

        if torch.sin(k_temp/2*cZ) == 0:
            Gz = nz**2
        else:  
            Gz = (torch.sin(k_temp/2*cZ*nz)/torch.sin(k_temp/2*cZ))**2

        I.append(F*Gx*Gy*Gz)
    return torch.tensor(I)

In [4]:
# read and parse a CIF file
file_path = "./data/test.cif"
cell_params,atom_type,coor = read_cif_file(file_path)
# In this test example, we actually built Zr4+, so we need to replace the Zr with Zr4+
atom_type = ['Zr4+' if i=='Zr' else i for i in atom_type]

# read in the coefficeint to calculate the scattering factor
name_path = './params/SF_name.txt'
file_path = './params/SF.xlsx'
sf_names,sf_factor = read_atomic_scattering_factor(name_path,file_path)

In [5]:
# define parameters for simulation
Xlambda = 1.54 # the wavelength of X-ray source in A
min_twotheta = 2.0 # the minimum value of 2-theta in degree
max_twotheta = 20.0 # the maximum value of 2-theta in degree
sampling_num = 1000 # sampling number
# define the crystal size 
nx = 1000
ny = 1000
nz = 1
# convert from degree to non-unit
twotheta = np.linspace(min_twotheta,max_twotheta,sampling_num)*np.pi/180 
# define the scattering vector
k = 4*np.pi*np.sin(twotheta/2)/Xlambda 
# define the scattering factor for individual atoms
fj = calculate_fj(k,atom_type,sf_names,sf_factor) 

# convert everything to pytorch
fj = torch.tensor(fj,dtype=torch.float64)
k = torch.tensor(k,dtype=torch.float64)
coor = torch.tensor(coor,dtype=torch.float64)

In [6]:
# create the basic vector of a,b and c according to the lattice constant 
# we force the lattice a on the (X,Y) surface
lattice_a = np.array([cell_params[0],0,0])
# we force the lattice b on the (X,Y) surface with a angle of 
lattice_b = np.array([cell_params[1]*np.cos(cell_params[5]),cell_params[1]*np.sin(cell_params[5]),0])
# we define the lattice c according to its angle between a and b
# solve the equation for c
lattice_c = (cell_params[2]*solve_lattice_c(lattice_a,lattice_b))
# the Z axis
Z = torch.tensor([0,0,1]).reshape(-1,1)

lattice_a = torch.tensor(lattice_a)
lattice_b = torch.tensor(lattice_b)
lattice_c = torch.tensor(lattice_c)

In [7]:
model = torchquad.MonteCarlo()
Is = []
for k_index in range(len(k)):
    k_temp = k[k_index]
    fj_temp = fj[k_index]
    integral_value = model.integrate(calculate_intensity,dim=2,N=10000,integration_domain = [[0,2*np.pi],[0,2*np.pi]])
    Is.append(integral_value)
plt.plot(twotheta*180/np.pi,torch.tensor(Is).cpu())
np.savetxt('./data/MonteCarlo.csv',np.array(torch.tensor(Is).cpu()),delimiter=',')

18:53:03|TQ-INFO| Computed integral was tensor(-9.5320e+09, dtype=torch.float64)
18:53:25|TQ-INFO| Computed integral was tensor(-5.6775e+09, dtype=torch.float64)
18:53:46|TQ-INFO| Computed integral was tensor(1.1233e+09, dtype=torch.float64)
18:54:07|TQ-INFO| Computed integral was tensor(1.9831e+11, dtype=torch.float64)
18:54:29|TQ-INFO| Computed integral was tensor(-7.4777e+09, dtype=torch.float64)
18:54:49|TQ-INFO| Computed integral was tensor(5.3349e+10, dtype=torch.float64)
18:55:16|TQ-INFO| Computed integral was tensor(4.9633e+10, dtype=torch.float64)
18:55:42|TQ-INFO| Computed integral was tensor(4.1688e+09, dtype=torch.float64)
18:56:12|TQ-INFO| Computed integral was tensor(-1.1636e+10, dtype=torch.float64)
18:56:47|TQ-INFO| Computed integral was tensor(7.9625e+10, dtype=torch.float64)
18:58:16|TQ-INFO| Computed integral was tensor(1.4182e+10, dtype=torch.float64)
18:59:55|TQ-INFO| Computed integral was tensor(3.2906e+10, dtype=torch.float64)
19:01:22|TQ-INFO| Computed integral 

KeyboardInterrupt: 